# Ekstraksi dan Pengolahan Data Kepemilikan Tanah

## Proker 1: Strukturisasi Data dan Arsip Digital Dokumen Pemilikan Tanah

### 1. Menyiapkan coding environment, termasuk menginstall dan/atau memperbarui library

In [ ]:
# Menginstall library-library
%pip install pandas
%pip install pytesseract
%pip install pdf2image
%pip install pillow
%pip install numpy
%pip install matplotlib
%pip install tensorflow
%pip install opencv-python

In [2]:
# Mengimport library-library untuk digunakan
import cv2
import numpy as np
import os
import pandas as pd
import pytesseract
from pdf2image import convert_from_path
from PIL import Image
from matplotlib import pyplot as plt
import re

### 2. Trial and Error halaman 1 dari file 'berkas 500-600.pdf'

##### Konversi PDF ke PNG 

In [ ]:
# Buat subdirektori untuk menampung gambar di dalam subdirektori "BERKAS LETTER C"
output_folder = 'file png berkas 500-600'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Konversi PDF ke gambar
pdf_path = 'BERKAS LETTER C/berkas 500-600.pdf'
images = convert_from_path(pdf_path)

# Simpan setiap gambar ke folder output
for i, image in enumerate(images):
    image_path = os.path.join(output_folder, f'halaman_{i+1}.png')
    image.save(image_path, 'PNG')

# Menampilkan jalur gambar yang disimpan
image_paths = [os.path.join(output_folder, f'halaman_{i+1}.png') for i in range(len(images))]
print("Gambar disimpan di:")
for path in image_paths:
    print(path)

In [4]:
# Muat gambar
image_path = 'file png berkas 500-600/halaman_1.png'
image = cv2.imread(image_path, cv2.IMREAD_COLOR)

# Cek jika gambar berhasil dimuat
if image is None:
    raise ValueError(f"Gambar tidak dimuat. Periksa apakah path sudah benar: {image_path}")

##### Upaya praproses gambar 1, **gagal**

Dicoba menggunakan:

- **Tesseract OCR**: Digunakan untuk mengekstrak teks dari gambar.
- **Praproses Gambar**:
- **Thresholding**: Menerapkan ambang batas adaptif untuk mengubah gambar ke format biner.
- **Penghapusan Derau**: Menggunakan operasi morfologi untuk menghilangkan derau dari gambar.
- **Deteksi Kontur**: Mendeteksi kontur untuk mengidentifikasi struktur tabel dalam gambar.
- **Deteksi Garis Tabel**: Menyorot garis tabel untuk meningkatkan akurasi OCR.

Meskipun telah melakukan upaya ini, dihadapi tantangan karena kualitas gambar pindaian dan teks tulisan tangan yang buruk, yang mengakibatkan kemajuan yang berarti terbatas.

In [ ]:
# Langkah 1: Terapkan thresholding adaptif dengan garis yang lebih tebal
'''
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # Konversi ke grayscale
adaptive_binary = cv2.adaptiveThreshold(
    gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 1001, 20
)

# Simpan output untuk langkah ini
step_1_path = os.path.join(output_dir, 'Step 1 - Thickened Adaptive Thresholded Image.png')
cv2.imwrite(step_1_path, adaptive_binary)

print(f"Langkah 1 selesai. Citra ambang batas adaptif yang dipertebal disimpan di: {step_1_path}")
'''

In [ ]:
# Langkah 2: Hapus area tangan
'''
# Sesuaikan kotak pembatas untuk menutupi tangan
hand_region = (0, adaptive_binary.shape[0] - 800, 600, 1000)  # x, y, width, height
x, y, w, h = hand_region

# Tutupi tangan dengan mengecat bagian tersebut dengan warna putih
hand_removed = adaptive_binary.copy()
cv2.rectangle(hand_removed, (x, y), (x + w, y + h), (255, 255, 255), -1)

# Simpan gambar yang sudah dibersihkan
step_2_path = os.path.join(output_dir, 'Step 2 - Hand Removed.png')
cv2.imwrite(step_2_path, hand_removed)

print(f"Langkah 2 selesai. Gambar yang diambil dengan tangan disimpan di: {step_2_path}")
'''

In [ ]:
# Langkah 3: Sorot garis tabel dengan warna hijau
'''
try:
    # Mendeteksi tepi menggunakan Canny
    edges = cv2.Canny(hand_removed, 500, 1500)  # Sesuaikan ambang batas sesuai kebutuhan (ambang_batas rendah, ambang_batas tinggi)

    # Lebarkan tepian untuk memperkuat garis
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))  # Sesuaikan ukuran kernel untuk ketebalan garis
    dilated_edges = cv2.dilate(edges, kernel, iterations=1)

    # Filter tepi yang terdeteksi berdasarkan geometri (garis horizontal/vertikal panjang)
    contours, _ = cv2.findContours(dilated_edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    mask = np.zeros_like(edges) # Buat mask kosong
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        aspect_ratio = max(w, h) / min(w, h) if min(w, h) > 0 else 0
        # Keep only long horizontal or vertical lines
        if (w > 50 and h < 10) or (h > 50 and w < 10):  # Sesuaikan ambang batas sesuai kebutuhan
            cv2.drawContours(mask, [contour], -1, 255, thickness=cv2.FILLED)

    # Kombinasikan mask dengan tepian yang melebar
    filtered_lines = cv2.bitwise_and(dilated_edges, mask)

    # Ubah gambar skala abu-abu menjadi BGR untuk hamparan warna
    table_lines_colored = cv2.merge([hand_removed, hand_removed, hand_removed])  # Grayscale ke BGR

    # Hamparkan warna hijau pada garis tabel yang terdeteksi
    table_lines_colored[filtered_lines > 0] = [0, 255, 0]  # Green (0, 255, 0)

    # Simpan gambar yang dihasilkan
    step_3_path = os.path.join(output_dir, 'Step 3 - Filtered Green Table Lines.png')
    cv2.imwrite(step_3_path, table_lines_colored)

    print(f"Langkah 3 selesai. Gambar garis tabel hijau yang difilter disimpan di: {step_3_path}")

except Exception as e:
    print(f"Terjadi error pada Langkah 3: {e}")
'''

##### Upaya praproses gambar 2

In [8]:
# Tentukan path input dan folder output
image_path = 'file png berkas 500-600/halaman_1.png'
output_folder = 'file png berkas 500-600/Preprocessed halaman 1 stuff'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

Langkah 1: Praproses Gambar

In [ ]:
# Langkah 1.1. Konversi ke grayscale
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Simpan gambar grayscale
output_path = os.path.join(output_folder, 'halaman_1_gray.png')
cv2.imwrite(output_path, gray_image)

print(f"Gambar grayscale disimpan di:{output_path}")

In [ ]:
# Langkah 1.2. Meningkatkan kontras menggunakan CLAHE
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
enhanced_image = clahe.apply(gray_image)

# Simpan gambar kontras yang ditingkatkan
output_path = os.path.join(output_folder, 'halaman_1_contrast.png')
cv2.imwrite(output_path, enhanced_image)

print(f"Gambar kontras yang ditingkatkan disimpan di:{output_path}")

In [ ]:
# Langkah 1.3. Hapus noise menggunakan median blurring
denoised_image = cv2.medianBlur(enhanced_image, 5)

# Simpan gambar yang sudah dihilangkan noise-nya
output_path = os.path.join(output_folder, 'halaman_1_denoised.png')
cv2.imwrite(output_path, denoised_image)

print(f"Gambar yang sudah dihilangkan noisenya disimpan di:{output_path}")

In [ ]:
# Langkah 1.4. Normalisasi dimensi
# Tentukan dimensi target
target_width, target_height = 1024, 1024

# Ubah ukuran gambar sambil mempertahankan rasio aspek
h, w = denoised_image.shape
scaling_factor = min(target_width / w, target_height / h)
new_width = int(w * scaling_factor)
new_height = int(h * scaling_factor)

# Ubah ukuran gambar
resized_image = cv2.resize(denoised_image, (new_width, new_height), interpolation=cv2.INTER_AREA)

# Sesuaikan gambar dengan dimensi target
delta_w = target_width - new_width
delta_h = target_height - new_height
top, bottom = delta_h // 2, delta_h - (delta_h // 2)
left, right = delta_w // 2, delta_w - (delta_w // 2)
normalized_image = cv2.copyMakeBorder(resized_image, top, bottom, left, right, cv2.BORDER_CONSTANT, value=255)

# Simpan gambar yang dinormalisasi
output_path = os.path.join(output_folder, 'halaman_1_normalized.png')
cv2.imwrite(output_path, normalized_image)

print(f"Gambar yang dinormalisasi disimpan di: {output_path}")

Langkah 2: Menghilangkan tabel

In [ ]:
# Langkah 2.1: Mendeteksi Struktur Tabel

# Threshold gambar ke biner
_, binary = cv2.threshold(normalized_image, 200, 255, cv2.THRESH_BINARY_INV)

# Mendeteksi garis horizontal
horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (50, 1))
horizontal_lines = cv2.morphologyEx(binary, cv2.MORPH_OPEN, horizontal_kernel)

# Mendeteksi garis vertikal
vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 50))
vertical_lines = cv2.morphologyEx(binary, cv2.MORPH_OPEN, vertical_kernel)

# Gabungkan garis horizontal dan vertikal
table_structure = cv2.add(horizontal_lines, vertical_lines)

# Dilate untuk menggabungkan celah pada garis tabel
table_structure = cv2.dilate(table_structure, np.ones((3, 3), np.uint8))

# Simpan gambar struktur tabel yang dihasilkan
output_path = 'BERKAS LETTER C/file png berkas 500-600/Preprocessed halaman 1 stuff/halaman_1_table_structure.png'
cv2.imwrite(output_path, table_structure)

# Print konfirmasi dan tampilkan output yang disimpan
print(f"Table structure image saved at: {output_path}")


In [ ]:
# Langkah 2.2: Hapus Struktur Tabel untuk Mengekstrak Teks

# Normalisasikan struktur tabel ke biner (0 dan 255)
_, table_structure_binary = cv2.threshold(table_structure, 127, 255, cv2.THRESH_BINARY)

# Buat mask di tempat garis tabel berada
table_lines_mask = table_structure_binary

# Ganti garis tabel pada gambar yang dinormalisasikan dengan warna putih
text_only_image = cv2.add(normalized_image, table_lines_mask)

# Simpan gambar yang hanya berupa teks yang dihasilkan
output_path = 'BERKAS LETTER C/file png berkas 500-600/Preprocessed halaman 1 stuff/halaman_1_text_only.png'
cv2.imwrite(output_path, text_only_image)

# Cetak konfirmasi dan simpan output
print(f"Gambar hanya teks disimpan di:{output_path}")

Langkah 3: Melakukan OCR

In [ ]:
# Langkah 3.1: Terapkan OCR untuk Mengekstrak Teks
# Tentukan konfigurasi OCR
custom_config = r'--psm 13 -c tessedit_char_whitelist="0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ.,() "'

# Terapkan OCR pada gambar teks saja
ocr_result = pytesseract.image_to_string(text_only_image, lang='eng', config='custom_config')

# Simpan output OCR ke file teks
output_path = 'file png berkas 500-600/Preprocessed halaman 1 stuff/halaman_1_ocr_output.txt'
with open(output_path, 'w', encoding='utf-8') as f:
    f.write(ocr_result)

# Print konfirmasi dan hasil OCR
print(f"Hasil OCR disimpan di: {output_path}")
print("Output OCR: ")
print(ocr_result)

In [ ]:
# Langkah 3.2: Bersihkan hasil OCR
cleaned_ocr = re.sub(r'[^a-zA-Z0-9\s.,]', '', ocr_result)  # Hanya simpan alfanumerik dan tanda baca dasar
cleaned_ocr = re.sub(r'\s+', ' ', cleaned_ocr).strip()  # Hapus spasi ekstra dan rapikan

# Simpan output OCR yang telah dibersihkan ke dalam file teks
output_path_cleaned = 'file png berkas 500-600/Preprocessed halaman 1 stuff/halaman_1_ocr_cleaned.txt'
with open(output_path_cleaned, 'w', encoding='utf-8') as f:
    f.write(cleaned_ocr)

# Cetak konfirmasi dan output OCR yang telah dibersihkan
print(f"Hasil OCR yang dibersihkan disimpan di: {output_path_cleaned}")
print("Output OCR yang dibersihkan:")
print(cleaned_ocr)

In [ ]:
# Langkah 3.3: Simpan Output OCR dengan Penempatan Header yang Tepat

# Tentukan ID halaman dan tajuk
header = ["OCR Output"]
page_id = "page_500"

# Membagi OCR yang telah dibersihkan menjadi kata-kata individual
words = [page_id] + cleaned_ocr.split()

# Buat DataFrame dengan kata-kata sebagai satu baris
df = pd.DataFrame([words])

# Perbarui header secara manual
df.columns = header + ["" for _ in range(len(words) - 1)]

# Simpan DataFrame ke CSV
csv_output_path = 'file png berkas 500-600/Preprocessed halaman 1 stuff/halaman_1_ocr_paginated_final_header.csv'
df.to_csv(csv_output_path, index=False, header=False)

# Print konfirmasi dan pratinjau
print(f"Hasil OCR halaman akhir disimpan ke CSV di:{csv_output_path}")
print("Pratinjau CSV:")
print(df.head())

## Proker 2: Alat Pencarian Dokumen Kepemilikan Tanah

### 3. Memproses Semua Halaman dengan Pipeline OCR

In [ ]:
# Langkah 1: Mengonversi PDF ke Gambar dan Mengatur Folder Penyimpanan

# Daftar file PDF dan folder keluaran yang sesuai
pdf_files = [
    ('BERKAS LETTER C/berkas 0-100.pdf', 'file png berkas 0-100'),
    ('BERKAS LETTER C/berkas 100-200.pdf', 'file png berkas 100-200'),
    ('BERKAS LETTER C/berkas 200-300.pdf', 'file png berkas 200-300'),
    ('BERKAS LETTER C/berkas 300-400.pdf', 'file png berkas 300-400'),
    ('BERKAS LETTER C/berkas 400-500.pdf', 'file png berkas 400-500'),
    ('BERKAS LETTER C/berkas 500-600.pdf', 'file png berkas 500-600'),
    ('BERKAS LETTER C/berkas 600-700.pdf', 'file png berkas 600-700'),
    ('BERKAS LETTER C/berkas 700-800.pdf', 'file png berkas 700-800'),
    ('BERKAS LETTER C/berkas 800-900.pdf', 'file png berkas 800-900'),
    ('BERKAS LETTER C/berkas 900-968.pdf', 'file png berkas 900-968'),
    ('BERKAS LETTER C/berkas sisa sobekan 400-500.pdf', 'file png berkas sisa sobekan 400-500')
]


# Proses setiap file PDF
for pdf_path, output_folder in pdf_files:
    # Pastikan folder output ada
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    print(f"Mengonversi {pdf_path} ke gambar...")
    
    # Konversi PDF ke gambar
    images = convert_from_path(pdf_path)
    for i, image in enumerate(images):
        image_path = os.path.join(output_folder, f'halaman_{i+1}.png')  # Tentukan nama file
        image.save(image_path, 'PNG')  # Simpan gambar dalam format PNG
        print(f"Gambar halaman {i+1} dari {pdf_path} disimpan di: {image_path}")  # Informasi gambar yang disimpan
    
    print(f"Konversi {pdf_path} selesai!")

print("Semua file PDF selesai diproses!")

In [20]:
# Langkah 2: Membuat pipeline untuk langkah-langkah pra-pemrosesan

# Daftar langkah
steps = [
    "langkah 1.1 konversi ke grayscale",
    "langkah 1.2 meningkatkan kontras menggunakan CLAHE",
    "langkah 1.3 menghapus noise menggunakan median blurring",
    "langkah 1.4 normalisasi dimensi",
    "langkah 2.1 mendeteksi struktur tabel",
    "langkah 2.2 menghapus struktur tabel untuk mengekstraksi teks",
    "langkah 3.1 menerapkan OCR untuk mengekstrak teks",
    "langkah 3.2 membersihkan hasil OCR",
    "langkah 3.3 menyimpan hasil OCR dengan header yang benar",
]

# Daftar semua folder file PNG
base_folders = [
    "file png berkas 0-100",
    "file png berkas 100-200",
    "file png berkas 200-300",
    "file png berkas 300-400",
    "file png berkas 400-500",
    "file png berkas 500-600",
    "file png berkas 600-700",
    "file png berkas 700-800",
    "file png berkas 800-900",
    "file png berkas 900-968",
    "file png berkas sisa sobekan 400-500",
]

# Fungsi untuk memproses semua folder
def process_all_folders(base_folders, steps):
    for base_folder in base_folders:
        print(f"Memproses folder: {base_folder}")

        # Buat subfolder untuk setiap langkah di dalam folder dasar
        output_folders = {step: os.path.join(base_folder, step) for step in steps}
        for folder in output_folders.values():
            os.makedirs(folder, exist_ok=True)

        # Jalankan proses untuk setiap gambar di folder
        process_images(base_folder, output_folders)

# Fungsi untuk memproses gambar di satu folder
def process_images(base_folder, output_folders):
    # Ambil daftar file gambar dari folder
    image_files = [f for f in os.listdir(base_folder) if f.endswith(".png")]
    
    for image_file in image_files:
        print(f"Memproses {image_file}...")
        image_path = os.path.join(base_folder, image_file)
        image = cv2.imread(image_path)

        # Langkah 1.1: Konversi ke grayscale
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        output_path = os.path.join(output_folders["langkah 1.1 konversi ke grayscale"], image_file)
        cv2.imwrite(output_path, gray_image)

        # Langkah 1.2: Tingkatkan kontras menggunakan CLAHE
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        contrast_image = clahe.apply(gray_image)
        output_path = os.path.join(output_folders["langkah 1.2 meningkatkan kontras menggunakan CLAHE"], image_file)
        cv2.imwrite(output_path, contrast_image)

        # Langkah 1.3: Hapus noise menggunakan median blurring
        denoised_image = cv2.medianBlur(contrast_image, 5)
        output_path = os.path.join(output_folders["langkah 1.3 menghapus noise menggunakan median blurring"], image_file)
        cv2.imwrite(output_path, denoised_image)

        # Langkah 1.4: Normalisasi dimensi
        target_width, target_height = 1024, 1024
        h, w = denoised_image.shape
        scaling_factor = min(target_width / w, target_height / h)
        new_width = int(w * scaling_factor)
        new_height = int(h * scaling_factor)
        resized_image = cv2.resize(denoised_image, (new_width, new_height), interpolation=cv2.INTER_AREA)
        delta_w = target_width - new_width
        delta_h = target_height - new_height
        top, bottom = delta_h // 2, delta_h - (delta_h // 2)
        left, right = delta_w // 2, delta_w - (delta_w // 2)
        normalized_image = cv2.copyMakeBorder(resized_image, top, bottom, left, right, cv2.BORDER_CONSTANT, value=255)
        output_path = os.path.join(output_folders["langkah 1.4 normalisasi dimensi"], image_file)
        cv2.imwrite(output_path, normalized_image)

        # Langkah 2.1: Deteksi struktur tabel
        _, binary = cv2.threshold(normalized_image, 200, 255, cv2.THRESH_BINARY_INV)
        horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (50, 1))
        horizontal_lines = cv2.morphologyEx(binary, cv2.MORPH_OPEN, horizontal_kernel)
        vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 50))
        vertical_lines = cv2.morphologyEx(binary, cv2.MORPH_OPEN, vertical_kernel)
        table_structure = cv2.add(horizontal_lines, vertical_lines)
        table_structure = cv2.dilate(table_structure, np.ones((3, 3), np.uint8))
        output_path = os.path.join(output_folders["langkah 2.1 mendeteksi struktur tabel"], image_file)
        cv2.imwrite(output_path, table_structure)

        # Langkah 2.2: Hapus struktur tabel untuk mengekstraksi teks
        _, table_structure_binary = cv2.threshold(table_structure, 127, 255, cv2.THRESH_BINARY)
        table_lines_mask = table_structure_binary
        text_only_image = cv2.add(normalized_image, table_lines_mask)
        output_path = os.path.join(output_folders["langkah 2.2 menghapus struktur tabel untuk mengekstraksi teks"], image_file)
        cv2.imwrite(output_path, text_only_image)

        # Langkah 3.1: Terapkan OCR
        custom_config = r'--psm 13 -c tessedit_char_whitelist="0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ.,() "'
        ocr_result = pytesseract.image_to_string(text_only_image, lang='eng', config='custom_config')
        output_path = os.path.join(output_folders["langkah 3.1 menerapkan OCR untuk mengekstrak teks"], image_file.replace(".png", ".txt"))
        with open(output_path, "w", encoding="utf-8") as f:
            f.write(ocr_result)

        # Langkah 3.2: Membersihkan hasil OCR
        cleaned_ocr = re.sub(r'[^a-zA-Z0-9\s.,]', '', ocr_result)
        cleaned_ocr = re.sub(r'\s+', ' ', cleaned_ocr).strip()
        output_path = os.path.join(output_folders["langkah 3.2 membersihkan hasil OCR"], image_file.replace(".png", ".txt"))
        with open(output_path, "w", encoding="utf-8") as f:
            f.write(cleaned_ocr)

    print("Semua langkah selesai untuk folder:", base_folder)

In [ ]:
# Langkah 3: Jalankan pipeline untuk semua folder
# Jalankan pipeline untuk semua folder
process_all_folders(base_folders, steps)

### 4. Menyimpan ke file CSV

In [ ]:
# Fungsi untuk menyimpan hasil OCR ke CSV
def save_ocr_results_to_csv(base_folder, output_folder):
    # Pastikan folder output ada
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Folder yang berisi hasil OCR (langkah 3.2)
    ocr_folder = os.path.join(base_folder, "langkah 3.2 membersihkan hasil OCR")
    ocr_files = [f for f in os.listdir(ocr_folder) if f.endswith(".txt")]

    # Dataframe untuk menyimpan semua hasil OCR
    ocr_data = []

    for ocr_file in ocr_files:
        file_path = os.path.join(ocr_folder, ocr_file)
        with open(file_path, "r", encoding="utf-8") as f:
            cleaned_text = f.read().strip()

        # Simpan data dalam format: Halaman | OCR Output
        page_name = ocr_file.replace(".txt", "")  # Nama file tanpa ekstensi
        ocr_data.append({"Page": page_name, "OCR Output": cleaned_text})

    # Konversi ke DataFrame dan simpan ke CSV
    ocr_df = pd.DataFrame(ocr_data)
    csv_output_path = os.path.join(output_folder, f"{base_folder.replace('file png ', '')}_OCR_results.csv")
    ocr_df.to_csv(csv_output_path, index=False, encoding="utf-8")

    print(f"Hasil OCR disimpan di: {csv_output_path}")

# Daftar folder utama dan jalankan proses untuk masing-masing
for base_folder in base_folders:
    print(f"Menyimpan hasil OCR untuk folder: {base_folder}")
    save_ocr_results_to_csv(
        base_folder=base_folder,
        output_folder=os.path.join(base_folder, "langkah 3.3 menyimpan hasil OCR dengan header yang benar")
    )